In [2]:
# ExcelToSql_2
#---------------------------------------------------#
# Importação das Bibliotecas                        #
#---------------------------------------------------#
import sqlalchemy as sa
from sqlalchemy import create_engine
import urllib.parse
import pyodbc as db
import pandas as pd
from pandas.io import sql
from pandas.core.frame import DataFrame
import openpyxl
from openpyxl import workbook, load_workbook

#---------------------------------------------------#
# Funções definidas pelo Usuário                     #
#---------------------------------------------------#
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
    except Error:
        print('Erro: ', error)

server = 'RICK-01-RICK\SQLEXPRESS'
database = 'Python'
username = ''
password = ''
constring = 'driver={SQL Server};server='+server+';database='+database+';Trusted_Connection=True'
sqlstring = 'mssql+pyodbc://@'+server+ '/' +database+ '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server'
#params = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};SERVER=RICK-01-RICK\SQLEXPRESS;DATABASE=Python;Trusted_Connection=yes")
params = urllib.parse.quote_plus('DRIVER={SQL Server Native Client 11.0};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes')

# Conexão com o Banco de Dados
try:
#    sql_conn = db.connect(constring)
#    engine = create_engine(sqlstring)
    engine = sa.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
    print('Passed Connect Sql Server')
except:
    print('Failed Connect Sql Server!')

# Monta o DataFrame do Excel
try:
    df = pd.read_excel('Tickers.xlsx', engine = 'openpyxl')
    print('Passed reading Excel')
except:
    print('Failed reading Excel!')

# Leitura sequencial da lista de Tickers Sql Server
result = engine.execute('SELECT * FROM Tickers')
for row in result:
    print(row)
print(engine.table_names())

# Leitura sequencial da lista de Tickers Sql Server
#rows = cursor.fetchall()
#print(' Codigo        Empresa')
#for i in rows:
#    print(i.Codigo, i.Empresa)
#print('***** Imprimiu Sql Server *****')
    
#display(df)

# Leitura sequencial da lista de Tickers Excel
erros = 0
for i, row in df.iterrows():
#    print(row.codigo, row.empresa)
    try:
        query = f"""INSERT INTO Tickers (Codigo, Empresa) VALUES ('{row.codigo}', '{row.empresa}')"""
        execute_query(engine, query)
#        execute_query(sql_conn, query)
#        cursor.execute('INSERT INTO Tickers (Codigo, Empresa) VALUES (?, ?)', row.codigo, row.empresa)
#        cursor.execute(query)
    except:
        erros = erros + 1
        print( 'Failed writing Sql Server! ',erros)

# Grava no Sql Server
if erros == 0:
    engine.commit()
    print('Gravação do banco de dados do sucesso')
    
# Fecha a conexão Sql Server
#cursor.close()
engine.close()

ModuleNotFoundError: No module named 'workbook'